In [ ]:
!pip install razdel network pymorphy2[fast] nltk rouge==0.3.1
!pip install --upgrade datasets tqdm transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 8.2 MB 23.7 MB/s 
     |████████████████████████████████| 371 kB 69.0 MB/s 
  Created wheel for network: filename=network-0.1-py3-none-any.whl size=3153 sha256=1668ae35db5958e71559e23c65f6f1e2a8de367f7b2bfff28e5cd1b1bc5749b5
  Stored in directory: /root/.cache/pip/wheels/27/48/b5/cf24ed2baa2c4f96c2af7440e13bd5676a24a175fab14479ac
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=854309 sha256=b2003614f7b71069fbeb3c8abea2dc470a9dc4c542dcf9af103ebedc8e28a34a
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built network DAWG
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 31.3 MB/s 
     |█████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/nlp_tut/lesson14/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset

dataset_train = load_dataset('json', data_files= path + "ru_all_train.jsonl", split='train[:1%]')
dataset_test = load_dataset('json', data_files = path + 'ru_all_test.jsonl', split='train[:2%]')

Using custom data configuration default-ccb196b987a1b5e3


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ccb196b987a1b5e3/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


Using custom data configuration default-27cefb4e7d3edd50


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-27cefb4e7d3edd50/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


In [ ]:
dataset_train

Dataset({
    features: ['title', 'text', 'url', 'timestamp'],
    num_rows: 6162
})

In [ ]:
dataset_test

Dataset({
    features: ['title', 'text', 'url', 'timestamp'],
    num_rows: 685
})

In [ ]:
dataset_test['text'][0]

'Серийный TLX  не слишком далеко ушел в вопросах дизайна от предвосхитившего его концепт-кара Type S. Экстерьер атлетичный и стремительный, хотя и понравится не каждому, а салон еще более причудливый. Пока сложно судить о том, насколько хитросплетение поверхностей получилось эргономичным, но выглядит внутреннее пространство действительно интересно и необычно. 1 / 11 Детище премиального подразделения концерна Honda для североамериканского рынка вооружится новыми двигателями. На смену 206-сильной "четверке" объемом 2,4 л и 3,5-литровому агрегату V6, развивающему 290 л.с., придут наддувные моторы — 2,0-литровый, выдающий 272 л.с. при 4800 об/мин и 380 Нм при 1600-4600 об/мин, а также V6 объемом 3,0 л для полноприводной спортверсии Type S, развивающий на 50% больше крутящего момента на низких оборотах чем современная "шестерка". По предварительным данным, он выдаст около 400 л.с. Обе модификации оснастят десятиступенчатыми автоматическими коробками передач. Впрочем, новинка должна восхитит

In [ ]:
dataset_test['title'][0]

'Рассекречен "убийца" BMW 5 Series от Acura'

In [ ]:
def len_tok(text):
  return len(text.split())

In [ ]:
max_len_sum, max_len_tl = max(map(len_tok, dataset_train['text'])), max(map(len_tok, dataset_train['title']))
max_len_sum, max_len_tl

(3454, 23)

In [ ]:
max_len_sum, max_len_tl = 100, 15

In [ ]:
model_name = 'IlyaGusev/rut5_base_sum_gazeta'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
  tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_sum)
  tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

  tokenized_input['labels'] = tokenized_label['input_ids']

  return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Parameter 'function'=<function tokenize at 0x7f1d90f04950> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/771 [00:00<?, ?ba/s]

  0%|          | 0/86 [00:00<?, ?ba/s]

In [ ]:
dataset_train.save_to_disk('titles/train') #(path + 'train')
dataset_test.save_to_disk('titles/test') #(path + 'test')

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

In [ ]:
output_dir = 'titles/output' #path + 'output'

training_args = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs=10,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    eval_accumulation_steps=1,
    prediction_loss_only=True,
    learning_rate=0.00001,
    evaluation_strategy='steps',
    save_steps=1000,
    save_total_limit=1,
    remove_unused_columns=True,
    run_name='titles',
    logging_steps=500,
    logging_first_step=False,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: timestamp, title, url, text. If timestamp, title, url, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6162
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7710
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:131: UserWarning: Creating a tensor from a list of numpy.ndarrays is ext

Step,Training Loss,Validation Loss
500,3.503000,2.553699
1000,2.850400,2.366058
1500,2.711900,2.253685
2000,2.560000,2.227347
2500,2.477100,2.147680
3000,2.435700,2.136686
3500,2.345300,2.112634
4000,2.305300,2.106324
4500,2.302000,2.084304
5000,2.276300,2.082257


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: timestamp, title, url, text. If timestamp, title, url, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 685
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: timestamp, title, url, text. If timestamp, title, url, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 685
  Batch size = 8
Saving model checkpoint to titles/output/checkpoint-1000
Configuration saved in titles/output/checkpoint-1000/config.json
Model weights saved in titles/output/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding ar

TrainOutput(global_step=7710, training_loss=2.448372146467909, metrics={'train_runtime': 2526.4254, 'train_samples_per_second': 24.39, 'train_steps_per_second': 3.052, 'total_flos': 8180738242560000.0, 'train_loss': 2.448372146467909, 'epoch': 10.0})

In [ ]:
trainer.save_model(output_dir + '/model')

Saving model checkpoint to titles/output/model
Configuration saved in titles/output/model/config.json
Model weights saved in titles/output/model/pytorch_model.bin


In [ ]:
trainer.save_model(path + 'model')

Saving model checkpoint to /content/drive/MyDrive/nlp_tut/lesson14/model
Configuration saved in /content/drive/MyDrive/nlp_tut/lesson14/model/config.json
Model weights saved in /content/drive/MyDrive/nlp_tut/lesson14/model/pytorch_model.bin


In [ ]:
INX = 50
print("TEXT: | {}".format(dataset_test['text'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

TEXT: | В течение последних десяти лет Intel почти не вносила изменений в конструкцию фирменных кулеров. Наконец, компания всё же взялась за штатную систему охлаждения, которой комплектуются десктопные процессоры 10-го поколения. Покупатели новинок уже опубликовали в сети первые снимки обновлённого аксессуара. Наиболее заметное изменение в конструкции штатной системы охлаждения — чёрный цвет вентилятора, радиатора и защитного кожуха, который к тому же стал толще и состоит из трёх опор вместо четырёх у предыдущей ревизии. Кулер сохранил круглую форму и по-прежнему оснащён алюминиевыми вытянутыми рёбрами с медным сердечником. Конструкция семилопастного вентилятора и система крепления видимых модификаций не претерпели. Наконец, четырёхконтактный кабель обзавёлся защитной оплёткой. По имеющейся информации, кулерами нового типа будут комплектоваться лишь процессоры без литеры K — предполагается, что покупатели оверклокерских моделей в любом случае предпочтут стороннюю систему охлаждения. Та

In [ ]:
device = "cuda"

In [ ]:
import torch

input_text = dataset_test['text'][INX]

with torch.no_grad():
  tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

  source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
  source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

  generated_ids = model.generate(
      input_ids = source_ids,
      attention_mask = source_mask,
      max_length=90,
      num_beams=7,
      temperature = 1.3,
      repetition_penalty=1,
      length_penalty=1,
      early_stopping=True,
      no_repeat_ngram_size=2
  )

  pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
pred

'В сети появились первые фото нового кулера Intel'